In [1]:
#Author: Anshul Kapoor
#Code Purpose: Make Single JSON for all EGamma SFs

#Names that you want for SFs for files above (same order please)
names=["Veto","Loose","Medium","Tight"]

#All the files for SFs
files={
    "Veto":"EGdata/egammaEffi.txt_Ele_Veto_postVFP_EGM2D.root",
    "Loose":"EGdata/egammaEffi.txt_Ele_Loose_postVFP_EGM2D.root",
    "Medium":"EGdata/egammaEffi.txt_Ele_Medium_postVFP_EGM2D.root",
    "Tight":"EGdata/egammaEffi.txt_Ele_Tight_postVFP_EGM2D.root",
}


#Names that you want for errors for files above (same order please)
nameJSON="egcorrs_withErrors_New.json" # Name of final JSON

sfhist="EGamma_SF2D"

In [2]:
from ROOT import TFile, TH1F, TCanvas, TString
import correctionlib.schemav1 as schema
from correctionlib.schemav1 import Correction
corrs=[]

#Function to extract SFs from EGamma standard root files
def getSFs(fn="filename",IsSF=1):
    tf = TFile(fn)
    fo = tf.Get(sfhist)
    Xbins=fo.GetNbinsX()
    Ybins=fo.GetNbinsY()
    X=[None]*(Xbins+1)
    Y=[None]*(Ybins+1)
    values=[]
    errors=[]
    for i in range(1,Xbins + 1):
        X[i-1]=(fo.GetXaxis().GetBinLowEdge(i))
    X[Xbins]=fo.GetXaxis().GetBinUpEdge(Xbins)
    for j in range(1,Ybins + 1):
        Y[j-1]=(fo.GetYaxis().GetBinLowEdge(j))
    Y[Ybins]=fo.GetYaxis().GetBinUpEdge(Ybins)
    for i in range(1,Xbins + 1):
        for j in range(1,Ybins + 1):
            values.append(fo.GetBinContent(i,j))
            errors.append(fo.GetBinError(i,j))
    if IsSF==1:
        valSFs=schema.MultiBinning.parse_obj({
            "nodetype": "multibinning",
            "edges": [
                X,
                Y,
            ],
            "content": values,
        })
        return valSFs
    if IsSF==0:
        valerrors=schema.MultiBinning.parse_obj({
            "nodetype": "multibinning",
            "edges": [
                X,
                Y,
            ],
            "content": errors,
        })
        return valerrors



Welcome to JupyROOT 6.22/06


In [3]:
corr = Correction.parse_obj(
    {
        "version": 1,
        "name": "ElectronSF",
        "inputs": [
            {"name": "SF or syst","type": "string"},
            {"name": "wp","type": "string"},
            {"type": "real", "name": "eta", "description": "possibly supercluster eta?"},
            {"name": "pt", "type": "real"},
        ],
        "output": {"name": "weight", "type": "real"},
        "data": schema.Category.parse_obj({
            "nodetype": "category",
            "keys": ["sf","syst"],
            "content": [
                schema.Category.parse_obj({
                    "nodetype": "category",
                    "keys": names,
                    "content":[
                        getSFs(fn=files[name],IsSF=1)
                        for name in names
                    ]
                }),
                schema.Category.parse_obj({
                    "nodetype": "category",
                    "keys": names,
                    "content":[
                        getSFs(fn=files[name],IsSF=0)
                        for name in names
                    ]
                }),
            ]
        })
    })
        
corrs.append(corr)

#Save JSON
from correctionlib.schemav1 import CorrectionSet
import gzip

cset = CorrectionSet.parse_obj({
    "schema_version": 1,
    "corrections": corrs,
})

with open(nameJSON, "w") as fout:
    fout.write(cset.json(exclude_unset=True, indent=4))

In [4]:
#Evaluator Example

import libcorrection

cset = libcorrection.CorrectionSet(nameJSON)

valsf= cset["ElectronSF"].evaluate("sf","Medium",1.1, 34.0)
print("sf is:"+str(valsf))

valsyst= cset["ElectronSF"].evaluate("syst","Medium",1.1, 34.0)
print("syst is:"+str(valsyst))


sf is:0.9858956336975098
syst is:0.014436147986067594
